In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.datasets import mnist
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import adam
import os

In [5]:
base_dir = 'images'



train_dir = os.path.join(base_dir,'train')


validation_dir = os.path.join(base_dir,'validation')


test_dir = os.path.join(base_dir,'test')

In [49]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

targetsize=96

batchsize=64

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(targetsize,targetsize),
        batch_size=batchsize,
        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
        validation_dir,
        target_size=(targetsize,targetsize),
        batch_size=batchsize,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        test_dir,
        target_size = (targetsize, targetsize),
        batch_size = batchsize,
        class_mode = 'categorical')

Found 3492 images belonging to 25 classes.
Found 1163 images belonging to 25 classes.
Found 1150 images belonging to 25 classes.


In [50]:
for data_batch, labels_batch in train_generator:
    print('data batch shape: ', data_batch.shape)
    print('labels batch shape: ', labels_batch.shape)
    break

data batch shape:  (64, 96, 96, 3)
labels batch shape:  (64, 25)


In [51]:
inputt,outputt = next(train_generator)

In [52]:
inputt.shape

(64, 96, 96, 3)

In [53]:
outputt.shape

(64, 25)

In [56]:
inputt2=np.reshape(inputt,(batchsize,targetsize*targetsize*3))

In [57]:
inputt2.shape

(64, 27648)

In [58]:
# def load_data():
#     (x_train, y_train), (x_test, y_test) = mnist.load_data()
#     x_train = (x_train.astype(np.float32) - 127.5)/127.5
    
#     # convert shape of x_train from (60000, 28, 28) to (60000, 784) 
#     # 784 columns per row
#     x_train = x_train.reshape(60000, 784)
#     return (x_train, y_train, x_test, y_test)
# (X_train, y_train,X_test, y_test)=load_data()
# print(X_train.shape)

def adam_optimizer():
    return adam(lr=0.0002, beta_1=0.5)

In [59]:
def create_generator():
    generator=Sequential()
    generator.add(Dense(units=512,input_dim=1000))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=targetsize*targetsize*3, activation='sigmoid'))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator

g=create_generator()
g.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 512)               512512    
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_38 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_39 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_52 (Dense)             (None, 27648)           

In [60]:
def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=2048,input_dim=targetsize*targetsize*3))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    
    discriminator.add(Dense(units=1024))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator

d =create_discriminator()
d.summary()



Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 2048)              56625152  
_________________________________________________________________
leaky_re_lu_40 (LeakyReLU)   (None, 2048)              0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_54 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
leaky_re_lu_41 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_55 (Dense)             (None, 512)             

In [61]:
 def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(1000,))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan
gan = create_gan(d,g)
gan.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
sequential_13 (Sequential)   (None, 27648)             29639680  
_________________________________________________________________
sequential_14 (Sequential)   (None, 1)                 59248641  
Total params: 88,888,321
Trainable params: 29,639,680
Non-trainable params: 59,248,641
_________________________________________________________________


In [62]:
def plot_generated_images(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 1000])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    #plt.savefig('gan_generated_image %d.png' %epoch)

In [63]:

def training(epochs=1, batch_size=batchsize):
    
    #Loading the data
    # batch_count = X_train.shape[0] / batch_size
    
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in range(int(3492/batch_size)):
            print(_/17.46,"%")
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 1000])
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            
            # get next group of values
            inputt,outputt = next(train_generator)
            inputt2=np.reshape(inputt,(batch_size,targetsize*targetsize*3))
            
            # Get a random set of  real images
            
            image_batch = inputt2
            
            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 1000])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
 
           
        plot_generated_images(e, generator)





In [64]:
training(2)

Epoch 1
0.0
0.000572737686139748
0.001145475372279496
0.001718213058419244
0.002290950744558992
0.0028636884306987398
0.003436426116838488
0.004009163802978236
0.004581901489117984
0.005154639175257732
0.0057273768613974796
0.006300114547537228
0.006872852233676976
0.0074455899198167235
0.008018327605956471
0.00859106529209622
0.009163802978235968
0.009736540664375716
0.010309278350515464
0.010882016036655211
0.011454753722794959
0.012027491408934709
0.012600229095074456
0.013172966781214204
0.013745704467353952
0.0143184421534937
0.014891179839633447
0.015463917525773196
0.016036655211912942
0.016609392898052692
0.01718213058419244
0.017754868270332187
0.018327605956471937


KeyboardInterrupt: 